In [1]:
import pandas as pd
import sqlite3

# Load the dataset
data = pd.read_csv('/kaggle/input/museum-directory/museums.csv')

# Create an SQLite connection
conn = sqlite3.connect('superstore.db')

# Load the data into an SQL table
data.to_sql('superstore', conn, if_exists='replace', index=False)

# Function to execute SQL queries
def execute_sql(query):
    return pd.read_sql_query(query, conn)

/tmp/ipykernel_34/1447821693.py:5: DtypeWarning: Columns (9,14,21) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/kaggle/input/museum-directory/museums.csv')


Which state has the museums with the highest average revenue? Highest expenses?

In [2]:
query = '''
SELECT `State (Administrative Location)` as State,
       AVG(Income) as Average_Income,
       AVG(Revenue) as Average_Revenue
FROM superstore
GROUP BY State
ORDER BY Average_Revenue DESC
LIMIT 1
'''
result = execute_sql(query)
print(result)

  State  Average_Income  Average_Revenue
0    DC    2.989774e+08     2.229338e+08


How many museums belong to universities?

In [ ]:
query = '''
SELECT COUNT(*) as University_Museums
FROM superstore
WHERE `Museum Type` LIKE '%University%'
'''
result = execute_sql(query)
print(result)

Categorize the states into Standard Federal Regions and answer which regions possess the most museums per capita:

In [11]:
regions_mapping = {
    'I': ['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 'Rhode Island', 'Vermont'],
    'II': ['New Jersey', 'New York', 'Puerto Rico', 'Virgin Islands'],
    'III': ['Delaware', 'District of Columbia', 'Maryland', 'Pennsylvania', 'Virginia', 'West Virginia'],
    'IV': ['Alabama', 'Florida', 'Georgia', 'Kentucky', 'Mississippi', 'North Carolina', 'South Carolina', 'Tennessee'],
    'V': ['Illinois', 'Indiana', 'Michigan', 'Minnesota', 'Ohio', 'Wisconsin'],
    'VI': ['Arkansas', 'Louisiana', 'New Mexico', 'Oklahoma', 'Texas'],
    'VII': ['Iowa', 'Kansas', 'Missouri', 'Nebraska'],
    'VIII': ['Colorado', 'Montana', 'North Dakota', 'South Dakota', 'Utah', 'Wyoming'],
    'IX': ['Arizona', 'California', 'Guam', 'Hawaii', 'Nevada'],
    'X': ['Alaska', 'Idaho', 'Oregon', 'Washington']
}

def state_to_region(state):
    state = state.title()  # Make sure the state name has the first letter capitalized and the rest lowercase
    for region, states in regions_mapping.items():
        if state in states:
            return region
    return None

data['Region'] = data['State (Administrative Location)'].apply(state_to_region)
data.to_sql('superstore', conn, if_exists='replace', index=False)

# Print the first 5 rows of the data DataFrame after updating the Region column
print(data.head())

# Load the state populations dataset
state_populations = pd.read_csv('/kaggle/input/us-state-populations-2018/State Populations.csv')

# Map the states to the regions
state_populations['Region'] = state_populations['State'].apply(state_to_region)

# Calculate the population for each region
region_populations = state_populations.groupby('Region')['2018 Population'].sum().reset_index()

# Save the region populations data as a new table in the SQLite database
region_populations.to_sql('region_population', conn, if_exists='replace', index=False)

# Execute the SQL query to find the region with the most museums per capita
query = '''
SELECT s.Region,
       COUNT(*) as Museum_Count,
       rp.`2018 Population` as Population,
       (COUNT(*) * 1.0 / rp.`2018 Population`) as Museums_Per_Capita
FROM superstore as s
JOIN region_population as rp
ON s.Region = rp.Region
GROUP BY s.Region
ORDER BY Museums_Per_Capita DESC
LIMIT 1
'''
result = execute_sql(query)
print(result)

    Museum ID                                        Museum Name  \
0  8400200098                    ALASKA AVIATION HERITAGE MUSEUM   
1  8400200117                            ALASKA BOTANICAL GARDEN   
2  8400200153  ALASKA CHALLENGER CENTER FOR SPACE SCIENCE TEC...   
3  8400200143                ALASKA EDUCATORS HISTORICAL SOCIETY   
4  8400200027                             ALASKA HERITAGE MUSEUM   

                                          Legal Name Alternate Name  \
0                    ALASKA AVIATION HERITAGE MUSEUM            NaN   
1                        ALASKA BOTANICAL GARDEN INC            NaN   
2  ALASKA CHALLENGER CENTER FOR SPACE SCIENCE TEC...            NaN   
3                ALASKA EDUCATORS HISTORICAL SOCIETY            NaN   
4                    ALASKA AVIATION HERITAGE MUSEUM            NaN   

                                     Museum Type Institution Name  \
0                                 HISTORY MUSEUM              NaN   
1  ARBORETUM, BOTANICAL GA